## Développement du "Modèle sur mesure avancé" avec des essais sur au moins deux word embeddings différents et en gardant celui qui permet d’obtenir les meilleures performances. Nous utiliserons Glove et FastText.
-  Nous allons construire et entraîner un réseau de neurones pour chaque embedding, évaluer les performances et enregistrer les expérimentations avec MLFlow. 
-  C'est la colonne clean_text_embeddings qui sera utilisée pour les embeddings.

### Charger le fichier CSV nettoyé

In [47]:
# Verification version de tensorflow
import tensorflow as tf
print(tf.__version__)

2.17.0


In [48]:
import os
import pandas as pd

# Chemin relatif pour charger les données nettoyées
file_path = os.path.join("..", "data", "cleaned_data_with_text_for_models.csv")

# Chargement du DataFrame nettoyé
data = pd.read_csv(file_path, index_col=0)

# Vérification du contenu du DataFrame

In [49]:
# Réinitialiser l'index pour récupérer toutes les colonnes, y compris 'text' si elle est utilisée comme index
data = data.reset_index()

# Vérifier les colonnes disponibles après réinitialisation de l'index
print(data.columns)


Index(['text', 'clean_text_tfidf', 'clean_text_embeddings', 'clean_text_bert',
       'target'],
      dtype='object')


In [50]:
# Dataframe info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   text                   1600000 non-null  object
 1   clean_text_tfidf       1592857 non-null  object
 2   clean_text_embeddings  1596608 non-null  object
 3   clean_text_bert        1600000 non-null  object
 4   target                 1600000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 61.0+ MB


In [51]:
# Valeurs manquantes
print(data.isnull().sum())

text                        0
clean_text_tfidf         7143
clean_text_embeddings    3392
clean_text_bert             0
target                      0
dtype: int64


In [52]:
# Filtrer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
missing_embeddings = data[data['clean_text_embeddings'].isnull()]

# Afficher deux exemples pour analyser les raisons des valeurs manquantes
print(missing_embeddings[['text', 'clean_text_tfidf', 'clean_text_embeddings']].head(2))

             text clean_text_tfidf clean_text_embeddings
208    @mandayyy               NaN                   NaN
249  @mandayyy                 NaN                   NaN


In [53]:
# Supprimer les lignes avec des valeurs manquantes dans la colonne 'clean_text_embeddings'
data = data.dropna(subset=['clean_text_embeddings'])


In [54]:
# Vérifier les valeurs manquantes après suppression
print(data.isnull().sum())

text                        0
clean_text_tfidf         3751
clean_text_embeddings       0
clean_text_bert             0
target                      0
dtype: int64


### Étape 1 : Charger les embeddings GloVe et FastText
- Les liens pour télécharger les embeddings GloVe et FastText sont les suivants :
- https://nlp.stanford.edu/projects/glove/
- https://fasttext.cc/docs/en/english-vectors.html


In [55]:
import os
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

# Chemin relatif pour charger les fichiers d'embeddings
glove_path = os.path.join("..", "data", "glove.twitter.27B.100d.txt")
fasttext_path = os.path.join("..", "data", "crawl-300d-2M-subword.bin")

# Charger les embeddings GloVe
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Charger les embeddings FastText (format Facebook binaire)
fasttext_model = load_facebook_vectors(fasttext_path)


### Étape 2 : Créer une fonction pour générer une matrice d'embeddings
- Créer une matrice d'embeddings pour chaque modèle, basée sur le vocabulaire des données textuelles.

In [56]:
import numpy as np

# Fonction pour créer la matrice d'embeddings
def create_embedding_matrix(embedding_model, vocab, embedding_dim):
    embedding_matrix = np.zeros((len(vocab) + 1, embedding_dim))
    for word, i in vocab.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
        else:
            # Si le mot n'est pas trouvé dans les embeddings, laisser un vecteur de zéros
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix


### Étape 3 : Préparer le tokenizer et les données
- Avant de créer le réseau de neurones, nous devons tokeniser le texte et préparer les données.

In [57]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Tokenisation des textes nettoyés pour les word embeddings
num_words = 10000
max_sequence_length = 100

# Utiliser TextVectorization pour transformer les textes
tv_layer = tf.keras.layers.TextVectorization(
    max_tokens=num_words,
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adapter TextVectorization sur les textes
tv_layer.adapt(data['clean_text_embeddings'])

# Convertir les textes en séquences
X = tv_layer(data['clean_text_embeddings'])

# Convertir le Tensor en tableau NumPy
X = X.numpy()

# Préparer les labels
y = data['target'].values  # Assurez-vous que y est également un tableau NumPy

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 4. Cellule countvectorizer pour les embeddings GloVe et FastText (transformer les textes en vecteurs)
- Nous utiliserons les textes déjà nettoyés pour les word embeddings à partir de la colonne clean_text_embeddings.

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import numpy as np

# Sous-échantillonnage des données à 1 % pour tester rapidement le pipeline
data_sample = data.sample(frac=0.001, random_state=42)

# Utiliser CountVectorizer pour transformer les textes
num_words = 10000
vectorizer = CountVectorizer(max_features=num_words)

# Adapter et transformer les textes
X = vectorizer.fit_transform(data_sample['clean_text_embeddings'])

# Padding manuel des séquences pour une longueur maximale de 100
X_padded = hstack([X, np.zeros((X.shape[0], max(0, 100 - X.shape[1])))]).A

# Préparer les labels
y = data_sample['target'].values

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


### 5. Créer et entraîner les modèles avec GloVe et FastText
- Nous allons maintenant construire et entraîner les modèles avec les embeddings GloVe et FastText.

- a) Modèle de réseau de neurones :

In [59]:
import tensorflow as tf

# Utilisation de tf.keras pour construire le modèle
def build_model(embedding_matrix, input_length):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=embedding_matrix.shape[0],
                                        output_dim=embedding_matrix.shape[1],
                                        weights=[embedding_matrix],
                                        input_length=input_length,
                                        trainable=False))  # Ne pas entraîner les embeddings
    model.add(tf.keras.layers.LSTM(128, return_sequences=False))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Classification binaire
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


- b) Entraînement du modèle avec GloVe :

In [60]:
import numpy as np
import tensorflow as tf

# Créer la matrice d'embeddings pour GloVe
def create_embedding_matrix(glove_model, word_index, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in glove_model:
            embedding_vector = glove_model[word]
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Créer un tokenizer et l'ajuster sur les textes originaux
# Remplacez 'data_sample' par le DataFrame qui contient vos données textuelles
num_words = 10000  # Assurez-vous que ce nombre correspond à vos besoins
texts = data_sample['clean_text_embeddings']
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(texts)

In [61]:
# Créer la matrice d'embeddings pour GloVe
embedding_dim_glove = 100  # Taille des vecteurs d'embeddings pour GloVe
embedding_matrix_glove = create_embedding_matrix(glove_model, tokenizer.word_index, embedding_dim_glove)

# Construire et entraîner le modèle avec GloVe
model_glove = build_model(embedding_matrix_glove, input_length=100)
history_glove = model_glove.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5


C:\Users\pat\.conda\envs\P7te\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.4949 - loss: 0.6919 - val_accuracy: 0.5188 - val_loss: 0.6982
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 101s 3s/step - accuracy: 0.5720 - loss: 0.6811 - val_accuracy: 0.5344 - val_loss: 0.6916
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 102s 3s/step - accuracy: 0.5609 - loss: 0.6830 - val_accuracy: 0.5312 - val_loss: 0.6897
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.5303 - loss: 0.6849 - val_accuracy: 0.5281 - val_loss: 0.6899
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.5722 - loss: 0.6816 - val_accuracy: 0.5125 - val_loss: 0.6888


- c) Entraînement du modèle avec FastText :

In [62]:
# Créer la matrice d'embeddings pour FastText
embedding_dim_fasttext = 300  # Taille des vecteurs d'embeddings pour FastText
embedding_matrix_fasttext = create_embedding_matrix(fasttext_model, tokenizer.word_index, embedding_dim_fasttext)

# Construire et entraîner le modèle avec FastText
input_length = 100  # La longueur des séquences padding
model_fasttext = build_model(embedding_matrix_fasttext, input_length=input_length)

# Entraîner le modèle
history_fasttext = model_fasttext.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/5


C:\Users\pat\.conda\envs\P7te\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.4846 - loss: 0.6947 - val_accuracy: 0.5094 - val_loss: 0.6931
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 123s 3s/step - accuracy: 0.5419 - loss: 0.6897 - val_accuracy: 0.5031 - val_loss: 0.6899
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.5309 - loss: 0.6845 - val_accuracy: 0.5031 - val_loss: 0.6932
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 122s 3s/step - accuracy: 0.5385 - loss: 0.6824 - val_accuracy: 0.5031 - val_loss: 0.6921
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - accuracy: 0.5535 - loss: 0.6863 - val_accuracy: 0.5281 - val_loss: 0.6941


### 6. Enregistrer les expérimentations avec MLFlow

- a) Enregistrer les résultats pour GloVe :

In [70]:
import mlflow
import mlflow.keras

# Enregistrer les résultats pour GloVe
mlflow.start_run(run_name="GloVe Embedding")
mlflow.log_param("embedding", "GloVe")
mlflow.log_param("embedding_dim", embedding_dim_glove)
mlflow.log_metric("accuracy", model_glove.evaluate(X_test, y_test)[1])
mlflow.keras.log_model(model_glove, "model_glove")
mlflow.end_run()

UnsupportedModelRegistryStoreURIException:  Model registry functionality is unavailable; got unsupported URI 'D:\OC_IA\P7\OC_IA_P7_analyse_sentiments_deep_Learning\mlruns' for model registry data storage. Supported URI schemes are: ['', 'file', 'databricks', 'databricks-uc', 'uc', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql']. See https://www.mlflow.org/docs/latest/tracking.html#storage for how to run an MLflow server against one of the supported backend storage locations.

- b) Enregistrer les résultats pour FastText :

In [ ]:
# Enregistrer les résultats pour FastText
mlflow.start_run(run_name="FastText Embedding")
mlflow.log_param("embedding", "FastText")
mlflow.log_param("embedding_dim", embedding_dim_fasttext)
mlflow.log_metric("accuracy", model_fasttext.evaluate(X_test, y_test)[1])
mlflow.keras.log_model(model_fasttext, "model_fasttext")
mlflow.end_run()

### 7. Conclusion
- Ce code utilise le fichier cleaned_data_with_text_for_models.csv et applique les word embeddings GloVe et FastText. Nous avons modifié le code pour utiliser les textes déjà nettoyés dans la colonne clean_text_embeddings. Après avoir entraîné les modèles, nous enregistrons les résultats et les modèles dans MLFlow afin de comparer les performances des deux embeddings et de sélectionner le meilleur.